In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm


df = pd.read_excel("Account Summary.xlsx")
df.head()

,Date,Day,Adjusted_Gross&Sales,Net_Sales,HST,Counted_Cash,Foreign_Cash&Equiv.,Total_Cash,Debit,Tims_Gift_Card&Redeem,...,Scan_and_Pay&Amex,Scan_and_Pay&Timcard,Digital_Visa,Digital_Mastercard,Digital_AMEX,Digital_Tims&Gift Card,Rebates,Tims_Gift_Card&Load,Deposit,Over_ /_Short
0,2022-01-01,Sat,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,0,0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00
1,2022-01-02,Sun,1081.13,1060.40,89.41,437.00,0.0,437.00,452.24,175.05,...,0,0,4.03,11.8,0.0,3.95,0.0,25.0,1189.31,14.50
2,2022-01-03,Mon,1919.92,1868.58,147.47,785.65,0.0,785.65,699.54,311.70,...,0,0,0.00,0.0,0.0,0.00,0.0,0.0,1992.17,-23.88
3,2022-01-04,Tue,2540.98,2435.94,188.05,1004.80,0.0,1004.80,850.72,508.46,...,0,0,2.30,0.0,0.0,0.00,0.0,0.0,2570.93,-0.06
4,2022-01-05,Wed,2562.88,2464.21,206.06,1057.75,0.0,1057.75,783.30,478.72,...,0,0,4.11,8.5,0.0,5.79,0.0,8.4,2682.02,3.35


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     365 non-null    datetime64[ns]
 1   Day                      365 non-null    object        
 2   Adjusted_Gross&Sales     365 non-null    float64       
 3   Net_Sales                365 non-null    float64       
 4   HST                      365 non-null    float64       
 5   Counted_Cash             365 non-null    float64       
 6   Foreign_Cash&Equiv.      365 non-null    float64       
 7   Total_Cash               365 non-null    float64       
 8   Debit                    365 non-null    float64       
 9   Tims_Gift_Card&Redeem    365 non-null    float64       
 10  Visa                     365 non-null    float64       
 11  MasterCard               365 non-null    float64       
 12  American_Express         365 non-nul

In [18]:
df.columns

Index(['Date', 'Day', 'Adjusted_Gross&Sales', 'Net_Sales', 'HST',
       'Counted_Cash', 'Foreign_Cash&Equiv.', 'Total_Cash', 'Debit',
       'Tims_Gift_Card&Redeem', 'Visa', 'MasterCard', 'American_Express',
       'Discover', 'Scan_and_Pay&Mastercard', 'Scan_and_Pay&VISA',
       'Scan_and_Pay&Amex', 'Scan_and_Pay&Timcard', 'Digital_Visa',
       'Digital_Mastercard', 'Digital_AMEX', 'Digital_Tims&Gift Card',
       'Rebates', 'Tims_Gift_Card&Load', 'Deposit', 'Over_ /_Short'],
      dtype='object')

# Hypotheses Test:

## 1.Relationship Between Net Sales and Rebate Usage

**H₀: Rebates and discounts do not significantly influence net sales.**

**H₁: Higher rebate usage leads to increased net sales.**



#### "Rebates" represents discounts, cashback, or promotional offers that are applied to sales

In [26]:
# Two groups: with rebates and without rebates

with_rebates = df[df["Rebates"] > 0]["Net_Sales"]
without_rebates = df[df["Rebates"] == 0]["Net_Sales"]

# Mean Sales
mean_with_rebates = with_rebates.mean()
mean_without_rebates = without_rebates.mean()
print(f"Mean Sales with Rebates: {round(mean_with_rebates, 2)}")
print(f"Mean Sales without Rebates: {round(mean_without_rebates, 2)}")


# Independent t-test
t_stat, p_value = stats.ttest_ind(with_rebates, without_rebates, equal_var=False)
print(f"T-Test Results: t-statistic = {t_stat:.4f}, p-value = {p_value:.6f}")

# Interpretation
if p_value < 0.05:
    print("Reject the Null Hypothesis: Rebates significantly impact Net Sales.")
else:
    print("Fail to Reject the Null Hypothesis: No significant impact of rebates on Net Sales.")

Mean Sales with Rebates: 2510.4
Mean Sales without Rebates: 1976.54
T-Test Results: t-statistic = 4.9424, p-value = 0.000009
Reject the Null Hypothesis: Rebates significantly impact Net Sales.


**Observations:**

#### When rebates were applied, the average net sales were higher.

#### Without rebates, the net sales were significantly lower.

**T-Test Results:**

**t-statistic = 4.9424** → Indicates a strong difference between the two groups.

**p-value = 0.000009** → Since the p-value is much smaller than 0.05, the difference is statistically significant.

## 2.ANOVA Test: Impact of Digital Payments on Net Sales

**H₀ (Null Hypothesis): There is a significant difference in Net Sales across different Digital Payment levels (Visa, MasterCard, AMEX, Tim’s Gift Card, etc.)**

**H₁ (Alternative Hypothesis): No significant difference in Net Sales across different Digital Payment levels**

In [27]:
df.columns

Index(['Date', 'Day', 'Adjusted_Gross&Sales', 'Net_Sales', 'HST',
       'Counted_Cash', 'Foreign_Cash&Equiv.', 'Total_Cash', 'Debit',
       'Tims_Gift_Card&Redeem', 'Visa', 'MasterCard', 'American_Express',
       'Discover', 'Scan_and_Pay&Mastercard', 'Scan_and_Pay&VISA',
       'Scan_and_Pay&Amex', 'Scan_and_Pay&Timcard', 'Digital_Visa',
       'Digital_Mastercard', 'Digital_AMEX', 'Digital_Tims&Gift Card',
       'Rebates', 'Tims_Gift_Card&Load', 'Deposit', 'Over_ /_Short',
       'Total_Digital_Payments', 'Payment_Category'],
      dtype='object')

In [32]:
null_columns = df.isna().all()
print(null_columns)

Date                       False
Day                        False
Adjusted_Gross&Sales       False
Net_Sales                  False
HST                        False
Counted_Cash               False
Foreign_Cash&Equiv.        False
Total_Cash                 False
Debit                      False
Tims_Gift_Card&Redeem      False
Visa                       False
MasterCard                 False
American_Express           False
Discover                   False
Scan_and_Pay&Mastercard    False
Scan_and_Pay&VISA          False
Scan_and_Pay&Amex          False
Scan_and_Pay&Timcard       False
Digital_Visa               False
Digital_Mastercard         False
Digital_AMEX               False
Digital_Tims&Gift Card     False
Rebates                    False
Tims_Gift_Card&Load        False
Deposit                    False
Over_ /_Short              False
Total_Digital_Payments     False
Payment_Category           False
dtype: bool


In [33]:
df["Total_Digital_Payments"] = (df["Digital_Visa"] + df["Digital_Mastercard"] + df['Debit'] +
                                  df["Digital_AMEX"] + df["Digital_Tims&Gift Card"] + df['Digital_Visa'] + 
                                  df['MasterCard'] + df['Discover'] + df['Tims_Gift_Card&Load'] + df['Tims_Gift_Card&Redeem'] + 
                                  df['Scan_and_Pay&Amex'] + df['Scan_and_Pay&Mastercard'] + df['Scan_and_Pay&Timcard'] + 
                                  df['Scan_and_Pay&VISA'])

# Mean digital payments and sales
mean_digital_payments = df["Total_Digital_Payments"].mean()
print(f"Mean Total Digital Payments: {round(mean_digital_payments, 2)}")

Mean Total Digital Payments: 1506.19


In [34]:
# Categorizing Digital Payments into Low, Medium, and High usage
df["Payment_Category"] = pd.qcut(df["Total_Digital_Payments"], q=3, labels=["Low", "Medium", "High"])

# Grouping Net Sales by Payment Category
low_payments = df[df["Payment_Category"] == "Low"]["Net_Sales"]
medium_payments = df[df["Payment_Category"] == "Medium"]["Net_Sales"]
high_payments = df[df["Payment_Category"] == "High"]["Net_Sales"]

# ANOVA test
f_stat, p_value_anova = stats.f_oneway(low_payments, medium_payments, high_payments)
print(f"F-Statistic: {f_stat:.2f}")
print(f"ANOVA P-Value: {p_value_anova:.6f}")

# Interpretation
if p_value_anova < 0.05:
    print("Reject the Null Hypothesis: There is a significant difference in Net Sales across different Digital Payment levels.")
else:
    print("Fail to Reject the Null Hypothesis : No significant difference in Net Sales across different Digital Payment levels.")



F-Statistic: 334.49
ANOVA P-Value: 0.000000
Reject the Null Hypothesis: There is a significant difference in Net Sales across different Digital Payment levels.
